### Load Damagescanner package

In [1]:
import os
import numpy
import pandas
from damagescanner.core import VectorScanner 

data_path = '..'

### Read input data

In [2]:
inun_map = os.path.join(data_path,'data','inundation','inundation_map.tif')
landuse = os.path.join(data_path,'data','landuse','landuse.shp')

### Create dummy maximum damage dictionary and curves DataFrame

In [3]:
maxdam = {"grass":5,
        "forest":10,
        "orchard":50,
        "residential":200,
        "industrial":300,
        "retail":300,
        "farmland":10,
        "cemetery":15,
        "construction":10,
        "meadow":5,
        "farmyard":5,
        "scrub":5,
        "allotments":10,
        "reservoir":5,
        "static_caravan":100,
        "commercial":300}

In [4]:
curves = numpy.array(
            [[0,0],
            [50,0.2],
            [100,0.4],
            [150,0.6],
            [200,0.8],
            [250,1]])  
    
curves = numpy.concatenate((curves,
                            numpy.transpose(numpy.array([curves[:,1]]*(len(maxdam)-1)))),
                           axis=1)

curves = pandas.DataFrame(curves)
curves.columns = ['depth']+list(maxdam.keys())
curves.set_index('depth',inplace=True)    

### Run the RasterScanner

In [5]:
%%time
# run the VectorScanner and return the landuse map with damage values
loss_df = VectorScanner(landuse,inun_map,curves,maxdam)

Damage per object: 100%|██████████████████████████████████████████████████████████| 6615/6615 [00:31<00:00, 211.64it/s]


Wall time: 1min 12s
